In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,1632211.72,-22.80,-11.80,-6.76,-1.23,-0.78,0.17,-0.14,-0.01
3,-22.80,310144.09,-5.01,0.60,0.33,-0.05,-0.04,0.01,0.01
4,-11.80,-5.01,68463.88,0.19,0.36,-0.04,-0.05,-0.01,-0.00
5,-6.76,0.60,0.19,10787.16,-0.05,0.00,0.01,-0.02,-0.00
6,-1.23,0.33,0.36,-0.05,1926.62,0.03,0.01,0.00,0.00
7,-0.78,-0.05,-0.04,0.00,0.03,405.42,0.00,0.00,0.00
8,0.17,-0.04,-0.05,0.01,0.01,0.00,103.45,0.00,-0.00
9,-0.14,0.01,-0.01,-0.02,0.00,0.00,0.00,19.77,-0.00
10,-0.01,0.01,-0.00,-0.00,0.00,0.00,-0.00,-0.00,1.44


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00004,-0.00005,-0.00002,-0.00003,0.00001,-0.00002,-0.00001
3,-0.00003,1.00000,-0.00003,0.00001,0.00001,-0.00000,-0.00001,0.00000,0.00002
4,-0.00004,-0.00003,1.00000,0.00001,0.00003,-0.00001,-0.00002,-0.00001,-0.00000
5,-0.00005,0.00001,0.00001,1.00000,-0.00001,0.00000,0.00001,-0.00004,-0.00002
6,-0.00002,0.00001,0.00003,-0.00001,1.00000,0.00003,0.00001,0.00001,0.00000
7,-0.00003,-0.00000,-0.00001,0.00000,0.00003,1.00000,0.00001,0.00004,0.00000
8,0.00001,-0.00001,-0.00002,0.00001,0.00001,0.00001,1.00000,0.00000,-0.00000
9,-0.00002,0.00000,-0.00001,-0.00004,0.00001,0.00004,0.00000,1.00000,-0.00003
10,-0.00001,0.00002,-0.00000,-0.00002,0.00000,0.00000,-0.00000,-0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

38.28454531895315

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.96196428e+11 1.05259176e+10 5.02034652e+08 1.21289700e+07
 3.71832302e+05 1.54875942e+04 8.93403193e+02 2.43122825e+01
 2.95367554e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.996826e-01,-0.024629,-0.005230,-0.000826,-0.000147,-0.000031,-0.000008,-0.000002,-1.152284e-07
1,2.443724e-02,0.999135,-0.033256,-0.004950,-0.000882,-0.000186,-0.000048,-0.000009,-6.174706e-07
2,6.015841e-03,0.032967,0.999060,-0.027080,-0.004696,-0.000989,-0.000253,-0.000048,-3.517133e-06
3,1.101176e-03,0.005776,0.026697,0.998913,-0.036963,-0.007493,-0.001908,-0.000366,-2.676671e-05
4,2.348257e-04,0.001230,0.005574,0.036323,0.997714,-0.055053,-0.013321,-0.002537,-1.844568e-04
5,6.137572e-05,0.000320,0.001450,0.009209,0.053269,0.994283,-0.090681,-0.015873,-1.151274e-03
6,2.082053e-05,0.000110,0.000498,0.003156,0.017704,0.087801,0.990957,-0.099618,-6.851414e-03
7,6.131468e-06,0.000032,0.000144,0.000915,0.005112,0.024318,0.096990,0.992248,-7.361438e-02
8,8.913443e-07,0.000005,0.000021,0.000133,0.000744,0.003536,0.013860,0.072541,9.972626e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00031737, 0.00086459, 0.00094017, 0.00108734, 0.002286  ,
       0.00571679, 0.00904258, 0.00775229, 0.00273744])